In [1]:
import urllib.request
import os

import urllib.request：导入 urllib 模块，将用于下载文件。
import os ：导入 os 模块，用于确认文件是否存在。

In [2]:
url = "http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls"
filepath = "data/titanic3.xls"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url,filepath)
    print('downloaded:',result)

In [3]:
import numpy
import pandas as pd

In [4]:
all_df = pd.read_excel(filepath)

In [5]:
all_df[:5]

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON"


以上字段中 survival （是否生存）是label标签字段，也就是我们要预测的目标，其余是特征字段。  
survival（是否生存）：0=否，1=是  
pclass（舱等）：1=头等舱，2=二等舱，3=三等舱  
name（姓名）  
sex（性别）：female=女性，male=男性  
age（年龄）  
sibsp（手足或配偶也在船上数量）  
parch（双亲或子女也在船上数量）  
ticket（船票号码）  
fare（旅客费用）  
cabin（舱位号码）  
embarked（登录港口）：C=Cherbourg，Q=Queenstown，S=Southampton

字段中的ticket、cabin与survival（是否生存）无关，将其忽略。

In [6]:
cols = ['survived','name','pclass','sex','age','sibsp','parch','fare','embarked']
all_df = all_df[cols]

In [7]:
all_df[:2]

,survived,name,pclass,sex,age,sibsp,parch,fare,embarked
0,1,"Allen, Miss. Elisabeth Walton",1,female,29.0000,0,0,211.3375,S
1,1,"Allison, Master. Hudson Trevor",1,male,0.9167,1,2,151.5500,S


name：姓名字段在预测阶段会使用，但是训练时不需要，必须先删除。  
age：有几项数据的 age 字段是 null 值，所遇必须将 null 值改为平均值。  
fare：有几项数据的 fare 字段是 null 值，所以必须将 null 值改为平均值。  
sex：性别字段是文字，我们必须转换为 0 与 1。
embarked：分类特征字段有 3 个分类C、Q、S，必须使用 One-Hot Encoding进行转换。

使用drop方法删除 name字段。

In [8]:
from pandas import DataFrame as df

In [9]:
df = all_df.drop(['name'],axis=1)

In [10]:
all_df.isnull().sum()

survived      0
name          0
pclass        0
sex           0
age         263
sibsp         0
parch         0
fare          1
embarked      2
dtype: int64

### 将 age 字段 和 fare 字段为 null的数据替换成平均值
下面代码先使用 df['age'].mean() 和 df['fare'].mean()方法计算 age 字段和 fare 字段的平均值。然后使用 df['age'].fillna(age_mean) 和 df['fare'].fillna(fare_mean) 将 null 值转换成平均值。

In [11]:
age_mean = df['age'].mean()
fare_mean = df['fare'].mean()
df['age'] = df['age'].fillna(age_mean)
df['fare'] = df['fare'].fillna(fare_mean)

### 转换性别字段为 0 与 1

In [12]:
df['sex'] = df['sex'].map({'female':0,'male':1}).astype(int)

### 将 embarked 字段进行一位有效编码转换
Pandas 提供了一个很方便的方法进行一位有效编码转换，使用 get_dummies()传入以下参数。  
data：要转换的 DataFrame ，在此输入df。  
columns：要转换的字段，在此输入["embarked"]。

In [13]:
x_OneHot_df = pd.get_dummies(data=df,columns=["embarked"])

In [14]:
x_OneHot_df[:3]

,survived,pclass,sex,age,sibsp,parch,fare,embarked_C,embarked_Q,embarked_S
0,1,1,0,29.0000,0,0,211.3375,0,0,1
1,1,1,1,0.9167,1,2,151.5500,0,0,1
2,0,1,0,2.0000,1,2,151.5500,0,0,1


### DataFrame 转换为 Array

In [15]:
ndarray = x_OneHot_df.values

### 查看 ndarry 的 shape

In [16]:
ndarray.shape

(1309, 10)

从结果可知，ndarray共 1309 项 10 个字段。


### 查看 ndarray 的前两项数据

In [17]:
ndarray[:2]

array([[   1.    ,    1.    ,    0.    ,   29.    ,    0.    ,    0.    ,
         211.3375,    0.    ,    0.    ,    1.    ],
       [   1.    ,    1.    ,    1.    ,    0.9167,    1.    ,    2.    ,
         151.55  ,    0.    ,    0.    ,    1.    ]])

### 提取 features 与 label
ndarray共有二维，第一维是项数，第二维是字段。

In [18]:
Label = ndarray[:,0]
Features = ndarray[:,1:]

### 查看前两项的 label 和 features字段

In [19]:
Label[:2]

array([ 1.,  1.])

In [20]:
Features[:2]

array([[   1.    ,    0.    ,   29.    ,    0.    ,    0.    ,  211.3375,
           0.    ,    0.    ,    1.    ],
       [   1.    ,    1.    ,    0.9167,    1.    ,    2.    ,  151.55  ,
           0.    ,    0.    ,    1.    ]])

# 将 ndarray 特征字段进行标准化

导入 sklearn 的数据预处理模块

In [21]:
from sklearn import preprocessing

建立 MinMaxScaler标准化刻度 minmax_scale,我们将使用 preprocessing.MinMaxScaler 来进行标准化，需输入参数 feature_range设置标准化之后

In [22]:
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))

使用 minmax_scale.fit_transform 进行标准化

In [23]:
scaleFeatures = minmax_scale.fit_transform(Features)

查看标准化之后的特征字段前两项数据

In [24]:
scaleFeatures[:2]

array([[ 0.        ,  0.        ,  0.36116884,  0.        ,  0.        ,
         0.41250333,  0.        ,  0.        ,  1.        ],
       [ 0.        ,  1.        ,  0.00939458,  0.125     ,  0.22222222,
         0.2958059 ,  0.        ,  0.        ,  1.        ]])

# 将数据分为训练数据与测试数据

将数据以随机方式分为训练数据与测试数据

In [25]:
msk = numpy.random.rand(len(all_df)) < 0.8 # 按照 8;2 的比例使用 numpy.random.rand 产生 msk 
train_df = all_df[msk] # 产生训练数据，为全部数据的80%
test_df = all_df[~msk] # 产生测试数据，为全部数据的20%

显示训练数据与测试数据项数

In [26]:
print('total:',len(all_df),
      'train:',len(train_df),
      'test:',len(test_df))

total: 1309 train: 1061 test: 248


创建 PreprocessData 函数进行数据的预处理

In [27]:
def PreprocessData(raw_df):
    df = raw_df.drop(['name'],axis=1)
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    df['sex'] = df['sex'].map({'female':0,'male':1}).astype(int)
    x_OneHot_df = pd.get_dummies(data=df,columns=["embarked"])
    
    ndarray = x_OneHot_df.values
    Features = ndarray[:,1:]
    Label = ndarray[:,0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0,1))
    scaledFeatures = minmax_scale.fit_transform(Features)
    
    return scaledFeatures,Label

对训练数据与测试数据进行预处理

In [28]:
train_Features,train_Label = PreprocessData(train_df)
test_Features,test_Label = PreprocessData(test_df)

查看数据预处理后训练数据的标签字段

In [29]:
train_Label[:2]

array([ 1.,  1.])